In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import torch
from torch.utils.data import DataLoader

import json

from tqdm import tqdm

from my_imp.data import MyDataset, gen_image_transform, gen_bbox_transform
from my_imp.copied.scene_annotation import annotate_objects

In [19]:
img_size = 256

In [20]:
ans_dict_json = 'data/clevr/CLEVR_built_dictionaries.json'

image_root = 'data/clevr/train/images/'
vocab_json = 'data/clevr/train/vocab.json'
train_scenes_json = 'data/clevr/train/scenes.json'
train_questions_json = 'data/clevr/train/questions.json'

val_image_root = 'data/clevr/val/images/'
val_vocab_json = 'data/clevr/val/vocab.json'
val_scenes_json = 'data/clevr/val/scenes.json'
val_questions_json = 'data/clevr/val/questions.json'

dataset_args = {
    'train': {
        'scenes_json': train_scenes_json,
        'questions_json': train_questions_json,
        'image_root': image_root,
        'vocab_json': vocab_json,
    },
    'val': {
        'scenes_json': val_scenes_json,
        'questions_json': val_questions_json,
        'image_root': val_image_root,
        'vocab_json': val_vocab_json,
    },
}

image_transform = gen_image_transform(img_size)
bbox_transform = gen_bbox_transform(img_size)

In [21]:
split = 'val'

my_dataset = MyDataset(
    image_transform=image_transform,
    bbox_transform=bbox_transform,
    ans_dict_json=ans_dict_json,
    **dataset_args[split],
)

Loading answers from: "data/clevr/CLEVR_built_dictionaries.json"
Loading scenes from: "data/clevr/val/scenes.json".
Loading questions from: "data/clevr/val/questions.json".
Loading vocab from: "data/clevr/val/vocab.json".
Preparing scenes
15000/15000
Preparing questions
149991/149991


In [23]:
my_dataset[0]

{'image': tensor([[[-0.3027, -0.3027, -0.3027,  ..., -0.3541, -0.3541, -0.3369],
          [-0.3369, -0.3198, -0.3027,  ..., -0.3541, -0.3541, -0.3369],
          [-0.3198, -0.3198, -0.3027,  ..., -0.3541, -0.3541, -0.3541],
          ...,
          [-0.0629, -0.0629, -0.0629,  ...,  0.5364,  0.5364,  0.5193],
          [-0.0801, -0.0801, -0.0629,  ...,  0.5536,  0.5364,  0.5364],
          [-0.0801, -0.0629, -0.0629,  ...,  0.5364,  0.5364,  0.5364]],
 
         [[-0.1800, -0.1975, -0.1975,  ..., -0.2325, -0.2325, -0.2150],
          [-0.2150, -0.1975, -0.1800,  ..., -0.2325, -0.2325, -0.2325],
          [-0.1975, -0.1975, -0.1975,  ..., -0.2325, -0.2325, -0.2325],
          ...,
          [ 0.0476,  0.0476,  0.0651,  ...,  0.6254,  0.6254,  0.6254],
          [ 0.0301,  0.0301,  0.0476,  ...,  0.6429,  0.6254,  0.6254],
          [ 0.0301,  0.0476,  0.0476,  ...,  0.6254,  0.6254,  0.6254]],
 
         [[ 0.0431,  0.0256,  0.0256,  ..., -0.0092, -0.0092, -0.0092],
          [ 0.0082,

In [24]:
from my_imp.data import collate_fn

In [25]:
loader = DataLoader(my_dataset, batch_size=2, num_workers=2, shuffle=True, collate_fn=collate_fn)

In [29]:
batch = next(iter(loader))

In [31]:
batch[0][0].size()

torch.Size([2, 3, 256, 384])

In [27]:
for batch in tqdm(loader):
    pass

  2%|▏         | 1742/74996 [01:06<38:33, 31.67it/s] 

KeyboardInterrupt: 

In [134]:
batch = next(iter(loader))

In [143]:
print(batch[0][0].size())
print(batch[0][1].size())
print(batch[0][2].size())
print(batch[0][3].size())
print(batch[1].size())


torch.Size([2, 3, 256, 384])
torch.Size([2, 9, 4])
torch.Size([2])
torch.Size([2, 27])
torch.Size([2])


In [113]:
images = [d['image'] for d in batch]

In [114]:
torch.stack(images).size()

torch.Size([2, 3, 256, 384])

In [131]:
torch.IntTensor([d['answer'] for d in batch], )

tensor([2, 3], dtype=torch.int32)

In [119]:
from torch.nn.utils.rnn import pad_sequence

In [115]:
objects = [d['objects'] for d in batch]

In [123]:
pad_sequence(objects, batch_first=True)

tensor([[[247.2000, 132.8000, 289.6000, 177.6000],
         [260.0000,  73.6000, 322.4000, 145.6000],
         [ 27.2000,  60.8000,  86.4000, 129.6000],
         [175.2000,  61.6000, 204.8000,  91.2000],
         [103.2000,  48.8000, 153.6000, 109.6000],
         [182.4000,  26.4000, 224.0000,  79.2000],
         [ 56.0000,  83.2000, 132.8000, 165.6000]],

        [[151.2000, 137.6000, 230.4000, 239.2000],
         [245.6000,  91.2000, 312.0000, 169.6000],
         [196.0000,  72.8000, 218.4000, 100.8000],
         [107.2000,  60.8000, 154.4000, 108.8000],
         [  0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000]]])

In [101]:
torch.stack()

TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray

In [75]:
import sys

sys.path.insert(0, '../NSCL-PyTorch-Release/')

In [76]:
from nscl.datasets.datasets import NSCLDataset

In [78]:
NSCLDataset(val_scenes_json, val_questions_json, image_root=val_image_root, vocab_json=val_vocab_json, image_transform=img_transform)

10 23:18:09 Loading scenes from: "data/clevr/val/scenes.json".
10 23:18:11 Loading questions from: "data/clevr/val/questions.json".
Exception ignored in: <function _DataLoaderIter.__del__ at 0x126e03378>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 677, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 659, in _shutdown_workers
    w.join()
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/popen_fork.py", line 48, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/popen_fork.py", lin